In [0]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Sequential(nn.Conv2d(1,8,3, padding = 1), nn.BatchNorm2d(8), nn.Dropout(0.1))

        self.conv2 = nn.Sequential(nn.Conv2d(8,16,3, padding = 1), nn.BatchNorm2d(16), nn.Dropout(0.1))
        self.conv3 = nn.Sequential(nn.Conv2d(16,32,3, padding = 1), nn.BatchNorm2d(32), nn.Dropout(0.1))
        #self.conv4 = nn.Sequential(nn.Conv2d(32,64,3, padding = 1), nn.BatchNorm2d(64), nn.Dropout(0.1))

        self.pool1 = nn.MaxPool2d(2,2)
        self.conv5 = nn.Sequential(nn.Conv2d(32,8,1), nn.BatchNorm2d(8), nn.Dropout(0.1))

        self.conv6 = nn.Sequential(nn.Conv2d(8,16,3, padding = 1), nn.BatchNorm2d(16), nn.Dropout(0.1))
        self.conv7 = nn.Sequential(nn.Conv2d(16,16,3, padding = 1), nn.BatchNorm2d(16), nn.Dropout(0.1))
        #self.conv8 = nn.Sequential(nn.Conv2d(32,64,3, padding = 1), nn.BatchNorm2d(64), nn.Dropout(0.1))

        #self.pool2 = nn.MaxPool2d(2,2)
        #self.conv9 = nn.Sequential(nn.Conv2d(64,8,1), nn.BatchNorm2d(8), nn.Dropout(0.1))

        self.conv10 = nn.Sequential(nn.Conv2d(16,16,3), nn.BatchNorm2d(16), nn.Dropout(0.1))
        self.conv11 = nn.Sequential(nn.Conv2d(16,16,3), nn.BatchNorm2d(16), nn.Dropout(0.1))
        self.conv12 = nn.Sequential(nn.Conv2d(16,16,3), nn.BatchNorm2d(16), nn.Dropout(0.1))
        #self.conv13 = nn.Sequential(nn.Conv2d(32,32,3), nn.BatchNorm2d(32), nn.Dropout(0.1))

        #self.conv14 = nn.Conv2d(,16,3)
        #self.conv12 = nn.Conv2d(16,10,3)

        self.conv15 = nn.Conv2d(16,10,1)

    def forward(self, x):
        x = self.pool1(F.relu(self.conv3(F.relu(self.conv2(F.relu(self.conv1(x)))))))
        x = F.relu(self.conv7(F.relu(self.conv6(F.relu(self.conv5(x))))))
        x = F.relu(self.conv12(F.relu(self.conv11(F.relu(self.conv10(x))))))
        #x = F.relu(self.conv10(F.relu(self.conv9(x))))
        #x = F.relu(self.conv12(F.relu(self.conv11(F.relu(self.conv10(x))))))

        x = F.avg_pool2d(x,(6,6))
        
        x = self.conv15(x)
        x = x.view(-1, 10)

        return F.log_softmax(x)

In [66]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 28, 28]              80
       BatchNorm2d-2            [-1, 8, 28, 28]              16
           Dropout-3            [-1, 8, 28, 28]               0
            Conv2d-4           [-1, 16, 28, 28]           1,168
       BatchNorm2d-5           [-1, 16, 28, 28]              32
           Dropout-6           [-1, 16, 28, 28]               0
            Conv2d-7           [-1, 32, 28, 28]           4,640
       BatchNorm2d-8           [-1, 32, 28, 28]              64
           Dropout-9           [-1, 32, 28, 28]               0
        MaxPool2d-10           [-1, 32, 14, 14]               0
           Conv2d-11            [-1, 8, 14, 14]             264
      BatchNorm2d-12            [-1, 8, 14, 14]              16
          Dropout-13            [-1, 8, 14, 14]               0
           Conv2d-14           [-1, 16,

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:42: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [0]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [0]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
          test_loss, correct, len(test_loader.dataset),
          100. * correct / len(test_loader.dataset)))

In [69]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.015, momentum=0.9)

for epoch in range(1, 20):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)



  0%|          | 0/469 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:42: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


loss=2.347456932067871 batch_id=0:   0%|          | 0/469 [00:00<?, ?it/s]

loss=2.3584787845611572 batch_id=1:   0%|          | 0/469 [00:00<?, ?it/s]

loss=2.3584787845611572 batch_id=1:   0%|          | 2/469 [00:00<00:28, 16.30it/s]

loss=2.2942662239074707 batch_id=2:   0%|          | 2/469 [00:00<00:28, 16.30it/s]

loss=2.3553242683410645 batch_id=3:   0%|          | 2/469 [00:00<00:28, 16.30it/s]

loss=2.2826056480407715 batch_id=4:   0%|          | 2/469 [00:00<00:28, 16.30it/s]

loss=2.3245959281921387 batch_id=5:   0%|          | 2/469 [00:00<00:28, 16.30it/s]

loss=2.3245959281921387 batch_id=5:   1%|▏         | 6/469 [00:00<00:24, 18.84it/s]

loss=2.3320205211639404 batch_id=6:   1%|▏         | 6/469 [00:00<00:24, 18.84it/s]

loss=2.308446884155


Test set: Average loss: 0.1320, Accuracy: 9617/10000 (96%)





loss=0.04770373925566673 batch_id=0:   0%|          | 0/469 [00:00<?, ?it/s]

loss=0.1718059480190277 batch_id=1:   0%|          | 0/469 [00:00<?, ?it/s] 

loss=0.1718059480190277 batch_id=1:   0%|          | 2/469 [00:00<00:30, 15.56it/s]

loss=0.15362781286239624 batch_id=2:   0%|          | 2/469 [00:00<00:30, 15.56it/s]

loss=0.1000732034444809 batch_id=3:   0%|          | 2/469 [00:00<00:30, 15.56it/s] 

loss=0.13526952266693115 batch_id=4:   0%|          | 2/469 [00:00<00:30, 15.56it/s]

loss=0.12885314226150513 batch_id=5:   0%|          | 2/469 [00:00<00:30, 15.56it/s]

loss=0.12885314226150513 batch_id=5:   1%|▏         | 6/469 [00:00<00:25, 18.13it/s]

loss=0.10035805404186249 batch_id=6:   1%|▏         | 6/469 [00:00<00:25, 18.13it/s]

loss=0.07620961219072342 batch_id=7:   1%|▏         | 6/469 [00:00<00:25, 18.13it/s]

loss=0.11699260771274567 batch_id=8:   1%|▏         | 6/469 [00:00<00:25, 18.13it/s]

loss=0.10461090505123138 batch_id=9:   1%|▏         | 6/469 [00:00<00


Test set: Average loss: 0.0805, Accuracy: 9742/10000 (97%)





loss=0.029772575944662094 batch_id=0:   0%|          | 0/469 [00:00<?, ?it/s]

loss=0.03915834426879883 batch_id=1:   0%|          | 0/469 [00:00<?, ?it/s] 

loss=0.03915834426879883 batch_id=1:   0%|          | 2/469 [00:00<00:27, 16.82it/s]

loss=0.07685043662786484 batch_id=2:   0%|          | 2/469 [00:00<00:27, 16.82it/s]

loss=0.05393798276782036 batch_id=3:   0%|          | 2/469 [00:00<00:27, 16.82it/s]

loss=0.09172150492668152 batch_id=4:   0%|          | 2/469 [00:00<00:27, 16.82it/s]

loss=0.1187397912144661 batch_id=5:   0%|          | 2/469 [00:00<00:27, 16.82it/s] 

loss=0.1187397912144661 batch_id=5:   1%|▏         | 6/469 [00:00<00:23, 19.53it/s]

loss=0.032305873930454254 batch_id=6:   1%|▏         | 6/469 [00:00<00:23, 19.53it/s]

loss=0.047430191189050674 batch_id=7:   1%|▏         | 6/469 [00:00<00:23, 19.53it/s]

loss=0.07224981486797333 batch_id=8:   1%|▏         | 6/469 [00:00<00:23, 19.53it/s] 

loss=0.12279719859361649 batch_id=9:   1%|▏         | 6/469 [00:

...


Test set: Average loss: 0.0189, Accuracy: 9937/10000 (99%)





loss=0.006584659218788147 batch_id=0:   0%|          | 0/469 [00:00<?, ?it/s]

loss=0.014147790148854256 batch_id=1:   0%|          | 0/469 [00:00<?, ?it/s]

loss=0.014147790148854256 batch_id=1:   0%|          | 2/469 [00:00<00:28, 16.54it/s]

loss=0.04569646343588829 batch_id=2:   0%|          | 2/469 [00:00<00:28, 16.54it/s] 

loss=0.010433878749608994 batch_id=3:   0%|          | 2/469 [00:00<00:28, 16.54it/s]

loss=0.007689617574214935 batch_id=4:   0%|          | 2/469 [00:00<00:28, 16.54it/s]

loss=0.013771981000900269 batch_id=5:   0%|          | 2/469 [00:00<00:28, 16.54it/s]

loss=0.013771981000900269 batch_id=5:   1%|▏         | 6/469 [00:00<00:23, 19.31it/s]

loss=0.03014378994703293 batch_id=6:   1%|▏         | 6/469 [00:00<00:23, 19.31it/s] 

loss=0.016575388610363007 batch_id=7:   1%|▏         | 6/469 [00:00<00:23, 19.31it/s]

loss=0.04997226595878601 batch_id=8:   1%|▏         | 6/469 [00:00<00:23, 19.31it/s] 

loss=0.03185516968369484 batch_id=9:   1%|▏         | 6/4


Test set: Average loss: 0.0153, Accuracy: 9944/10000 (99%)





loss=0.003911182284355164 batch_id=0:   0%|          | 0/469 [00:00<?, ?it/s]

loss=0.027939118444919586 batch_id=1:   0%|          | 0/469 [00:00<?, ?it/s]

loss=0.027939118444919586 batch_id=1:   0%|          | 2/469 [00:00<00:26, 17.62it/s]

loss=0.02203402668237686 batch_id=2:   0%|          | 2/469 [00:00<00:26, 17.62it/s] 

loss=0.00812067836523056 batch_id=3:   0%|          | 2/469 [00:00<00:26, 17.62it/s]

loss=0.0692121610045433 batch_id=4:   0%|          | 2/469 [00:00<00:26, 17.62it/s] 

loss=0.04596433416008949 batch_id=5:   0%|          | 2/469 [00:00<00:26, 17.62it/s]

loss=0.04596433416008949 batch_id=5:   1%|▏         | 6/469 [00:00<00:22, 20.50it/s]

loss=0.002781011164188385 batch_id=6:   1%|▏         | 6/469 [00:00<00:22, 20.50it/s]

loss=0.04307011142373085 batch_id=7:   1%|▏         | 6/469 [00:00<00:22, 20.50it/s] 

loss=0.01059998944401741 batch_id=8:   1%|▏         | 6/469 [00:00<00:22, 20.50it/s]

loss=0.0037377551198005676 batch_id=9:   1%|▏         | 6/469 


Test set: Average loss: 0.0159, Accuracy: 9952/10000 (100%)

